In [1]:
import os 
import numpy as npimport pandas as pdfrom google.colab import drive
drive.mount('/content/drive')
os.chdir('/content/drive/MyDrive/curriculum/5001/Individualproject')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [146]:
train = pd.read_csv("train.csv")
test = pd.read_csv("test.csv")
train["date"] = pd.to_datetime(train["date"])
id = test['id']
train.drop(['id'], axis=1, inplace=True)
test["date"] = pd.to_datetime(test["date"])
test.drop(['id'], axis=1, inplace=True)

In [147]:
Y = train['speed']
def train_features(df):
  df["year"] = df["date"].dt.year
  df["month"] = df["date"].dt.month
  df["day"] = df["date"].dt.day
  df["hour"] = df["date"].dt.hour
  df["weekday"] = df["date"].dt.weekday
  train_date = df[["year","month","day","hour","weekday"]]
  return train_date

X = train_features(train)
pre = train_features(test)

In [148]:
from sklearn.preprocessing import OrdinalEncoder, OneHotEncoder
import xgboost as xgb
from sklearn.metrics import mean_squared_error
# feature processing
one_hot_enc = OneHotEncoder(handle_unknown='ignore')
one_hot_enc.fit(X["year"].values.reshape(-1, 1))
tmp_array1 = one_hot_enc.transform(X["year"].values.reshape(-1, 1)).toarray()
tmp_col_name1 = one_hot_enc.get_feature_names(['year_'])
tmp_df1 = pd.DataFrame(tmp_array1, columns=tmp_col_name1)

X = pd.concat([tmp_df1, X], axis=1)
X = X.drop('year',axis=1)
X.head()
#
one_hot_enc = OneHotEncoder(handle_unknown='ignore')
one_hot_enc.fit(pre["year"].values.reshape(-1, 1))
tmp_array2 = one_hot_enc.transform(pre["year"].values.reshape(-1, 1)).toarray()
tmp_col_name2 = one_hot_enc.get_feature_names(['year_'])
tmp_df2 = pd.DataFrame(tmp_array2, columns=tmp_col_name2)
column_2017 = {'year__2017': np.zeros(len(pre), dtype='float64')}
df1 = pd.DataFrame(column_2017)
pre = pd.concat([df1, tmp_df2, pre], axis=1)
pre = pre.drop('year',axis=1)

In [149]:
x_train, x_test, y_train, y_test = train_test_split(X, Y, test_size=0.2, random_state=0)
'''
####决策树回归####
from sklearn import tree
model_DecisionTreeRegressor = tree.DecisionTreeRegressor()
####线性回归####
from sklearn import linear_model
model_LinearRegression = linear_model.LinearRegression()
####SVM回归####
from sklearn import svm
model_SVR = svm.SVR()
####KNN回归####
from sklearn import neighbors
model_KNeighborsRegressor = neighbors.KNeighborsRegressor()
####随机森林回归####
from sklearn import ensemble
model_RandomForestRegressor = ensemble.RandomForestRegressor(n_estimators=20)#这里使用20个决策树
####Adaboost回归####
from sklearn import ensemble
model_AdaBoostRegressor = ensemble.AdaBoostRegressor(n_estimators=50)#这里使用50个决策树
####GBRT回归####
from sklearn import ensemble
model_GradientBoostingRegressor = ensemble.GradientBoostingRegressor(n_estimators=100)#这里使用100个决策树
####Bagging回归####
from sklearn.ensemble import BaggingRegressor
model_BaggingRegressor = BaggingRegressor()
####ExtraTree极端随机树回归####
from sklearn.tree import ExtraTreeRegressor
model_ExtraTreeRegressor = ExtraTreeRegressor()
'''
model = xgb.XGBRegressor(max_depth=10, eta=0.2, n_estimators=200)
model.fit(x_train, y_train)
predicted_y = model.predict(x_test)
print('Test Score:', mean_squared_error(y_test, predicted_y))

[16:06:54] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
Test Score: 15.282739455427171


In [151]:
test_speed = model.predict(pre)
column_id = pd.DataFrame({'id': id})
column_speed = pd.DataFrame({'speed': test_speed})
submit = pd.concat([column_id,  column_speed], axis=1)
submit.to_csv('submit.csv', index=None)